In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

os.environ['OMP_NUM_THREADS'] = '1'

df = pd.read_csv(r'D:\OneDrive\GA_DASH\data_v2.csv', sep=';')
df = df.drop(['Unnamed: 13', 'Unnamed: 14','Unnamed: 15','Date'], axis=1)

# removendo host names que não vão ser incluídos
hosts_to_remove = ['compras.cariocashopping.com.br', 'compras.shoppingleblon.com.br', 'boulevardcampos.com.br']
df = df[~df['Host name'].isin(hosts_to_remove)]

# agregando as contas de montes claros
df['Host name'] = df['Host name'].replace('www.montesclarosshopping.com.br', 'montesclarosshopping.com.br')

# Find the maximum number of splits
max_splits = df['Branding interest'].str.split('/').apply(len).max()

# Split the 'Branding interest' column into multiple columns
df_split = df['Branding interest'].str.split('/', expand=True)

# Rename the new columns
df_split.columns = [f'Interest{i+1}' for i in range(max_splits)]

# Join the dataframes
df = pd.concat([df, df_split], axis=1)

# Encontrar os valores únicos para variavel1
interest1 = df['Interest1'].value_counts()

# Encontrar os valores únicos para variavel2
interest2 = df['Interest2'].value_counts()

# Encontrar os valores únicos para variavel3
interest3 = df['Interest3'].value_counts()

Eventname = df[['Event name','Active users','Engaged sessions','Sessions','New users','Interest1','Host name','User age bracket']]

Eventname['Active users'] = Eventname['Active users'].fillna(0)

# Create a copy of the DataFrame
new_df = Eventname.copy()

# Initialize the label encoder
le = LabelEncoder()

# Apply label encoding to each object column
for col in new_df.columns:
    if new_df[col].dtype == 'object':
        new_df[col] = le.fit_transform(new_df[col])

new_df['key'] = new_df['Event name'].astype(str) + new_df['Interest1'].astype(str) + new_df['Host name'].astype(str) + new_df['User age bracket'].astype(str)
new_df = new_df.drop(columns=["Event name", "Interest1", "Host name", "User age bracket"])
new_df_groupby = new_df.groupby(["key"]).sum()

# Select only the numeric columns and fill NA values
df_numeric = new_df_groupby.select_dtypes(include=[np.number])
df_numeric = df_numeric.fillna(df_numeric.mean())

# Apply PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_numeric)

# Apply KMeans to the PCA result
# The number of clusters is 4
kmeans = KMeans(n_clusters=4, random_state=0).fit(df_pca)

# Add the cluster labels to the DataFrame
new_df_groupby['cluster'] = kmeans.labels_

# Calculate the Silhouette Score
score = silhouette_score(df_pca, kmeans.labels_)
print("Silhouette Score: ", score)

# Step 1: Reset the index of new_df_groupby
new_df_groupby_reset = new_df_groupby.reset_index()

# Step 2: Create a new DataFrame with only 'key' and 'cluster'
df_key_cluster = new_df_groupby_reset[['key', 'cluster']].copy()

# Merge new_df and df_key_cluster on 'key' using a left join
merged_df = pd.merge(new_df, df_key_cluster, on='key', how='left')
cluster = merged_df['cluster']

result = pd.concat([Eventname, cluster], axis=1)

# Filter the DataFrame for rows where 'cluster' is 0
result_cluster0 = result[result['cluster'] == 0]

# Create a dictionary mapping each unique 'Host name' to a unique 'store' name
store_dict = {host: f'store{idx+1}' for idx, host in enumerate(result['Host name'].unique())}

# Use the dictionary to replace the 'Host name' values
result['Host name'] = result['Host name'].map(store_dict)

result.to_excel('result.xlsx')

C:\Users\surze\AppData\Local\Temp\ipykernel_1316\2380617127.py:11: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\OneDrive\GA_DASH\data_v2.csv', sep=';')
C:\Users\surze\AppData\Local\Temp\ipykernel_1316\2380617127.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Eventname['Active users'] = Eventname['Active users'].fillna(0)


Silhouette Score:  0.8268350943124168


In [3]:
import plotly.express as px

# Create a DataFrame with the PCA results and the cluster labels
df_plot = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
df_plot['cluster'] = kmeans.labels_
df_plot['key'] = new_df_groupby.index

# Create a scatter plot of the PCA results, colored by the cluster labels
fig = px.scatter(df_plot, x='PC1', y='PC2', color='cluster', hover_data=['key'])

# Show the plot
fig.show()